## Oxparis Visualization Notebook
In this notebook one can load images and gt from the oxparis dataset as given by the POLD2 team (they were givin it by remi).
Possible actions in this Notebook:
- Load Images from the dataset and look at it and GT
- Run model on image and compare result with gt
- also detect lines

In [1]:
import torch
import flow_vis
import numpy as np
from matplotlib import pyplot as plt
from gluefactory.datasets import get_dataset

line_neighborhood = 5 # in px used to nortmalize/ denormalize df

Define utility functions

In [2]:
def get_flow_vis(df, ang, line_neighborhood=5):
    norm = line_neighborhood + 1 - np.clip(df, 0, line_neighborhood)
    flow_uv = np.stack([norm * np.cos(ang), norm * np.sin(ang)], axis=-1)
    flow_img = flow_vis.flow_to_color(flow_uv, convert_to_bgr=False)
    return flow_img

def normalize_df(df):
    return -torch.log(df / line_neighborhood + 1e-6)

def denormalize_df(df_norm):
    return torch.exp(-df_norm) * line_neighborhood

def visualize_img_with_gt(dset, num=5, offset=0, lim_kpoints=-1):
    idxs = list(range(offset, offset+num, 1))
    _, ax = plt.subplots(len(idxs), 4, figsize=(20, num*4))
    for i in idxs:
        elem = dset[i]
        af = elem["deeplsd_angle_field"]
        df = elem["deeplsd_distance_field"]
        hmap = elem["superpoint_heatmap"]
        orig_pt = elem["orig_points"]
        
        ax[i, 0].axis('off')
        ax[i, 0].set_title('Heatmap')
        ax[i, 0].imshow(hmap)

        ax[i, 1].axis('off')
        ax[i, 1].set_title('Distance Field')
        ax[i, 1].imshow(df)

        ax[i, 2].axis('off')
        ax[i, 2].set_title('Angle Field')
        ax[i, 2].imshow(get_flow_vis(df, af))

        ax[i, 3].axis('off')
        ax[i, 3].set_title('Original')
        ax[i, 3].imshow(elem["image"].permute(1,2,0))
        ax[i, 3].scatter(orig_pt[:lim_kpoints, 0], orig_pt[:lim_kpoints, 1], marker="o", color="red", s=3)
    plt.show()
    

In [3]:
dset_conf = {
            "reshape": [400, 400], # ex [800, 800]
            "load_features": {
                "do": True,
                "check_exists": True,
                "point_gt": {
                    "data_keys": ["superpoint_heatmap"],
                    "use_score_heatmap": False,
                },
                "line_gt": {
                    "data_keys": ["deeplsd_distance_field", "deeplsd_angle_field"],
                },
            },
            "debug": True
        }
oxpa_2 = get_dataset("oxford_paris_mini_1view_jpldd")(dset_conf)
ds = oxpa_2.get_dataset(split="train")

# load one test element
elem = ds[0]
print(f"Keys: {elem.keys()}")

# print example shapes
af = elem["deeplsd_angle_field"]
df = elem["deeplsd_distance_field"]
hmap = elem["superpoint_heatmap"]
orig_pt = elem["orig_points"]

print(f"AF: type: {type(af)}, shape: {af.shape}, min: {torch.min(af)}, max: {torch.max(af)}")
print(f"DF: type: {type(df)}, shape: {df.shape}, min: {torch.min(df)}, max: {torch.max(df)}")
print(f"KP-HMAP: type: {type(hmap)}, shape: {hmap.shape}, min: {torch.min(hmap)}, max: {torch.max(hmap)}, sum: {torch.sum(hmap)}")

/local/home/fmoeller/.local/lib/python3.10/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
[10/04/2024 01:18:51 gluefactory.datasets.base_dataset INFO] Creating dataset OxfordParisMiniOneViewJPLDD
[10/04/2024 01:18:51 gluefactory.datasets.oxford_paris_mini_1view_jpldd INFO] NUMBER OF IMAGES: 12000


DATASET OVERALL(NO-SPLIT) IMAGES: 12000


[10/04/2024 01:18:52 gluefactory.datasets.oxford_paris_mini_1view_jpldd INFO] NUMBER OF IMAGES WITH GT: 12000


Keys: dict_keys(['name', 'scales', 'image_size', 'transform', 'original_image_size', 'image', 'orig_points', 'superpoint_heatmap', 'deeplsd_distance_field', 'deeplsd_angle_field'])
AF: type: <class 'torch.Tensor'>, shape: torch.Size([400, 400]), min: 0.0, max: 3.1415913105010986
DF: type: <class 'torch.Tensor'>, shape: torch.Size([400, 400]), min: 0.40395185351371765, max: 4.999999523162842
KP-HMAP: type: <class 'torch.Tensor'>, shape: torch.Size([400, 400]), min: 0.0, max: 0.4347566068172455, sum: 225.98916625976562


## Run a model to get predictions

Define utility for visualisation and example calculation

In [4]:
def compute_and_visualize(img_data_with_gt, model, device):
    img_torch = img_data_with_gt["image"].to(device).unsqueeze(0)
    with torch.no_grad():
        output_model = model({"image": img_torch})
    jpldd_kpjhm = output_model['keypoint_and_junction_score_map'][0].cpu()
    jpldd_af = output_model['line_anglefield'][0].cpu()
    jpldd_df = output_model['line_distancefield'][0].cpu()
    jpldd_kp = output_model['keypoints'][0].cpu()
        
    af_gt = img_data_with_gt["deeplsd_angle_field"].cpu()
    df_gt = img_data_with_gt["deeplsd_distance_field"].cpu()
    hmap_gt = img_data_with_gt["superpoint_heatmap"].cpu()
    orig_pt_gt = img_data_with_gt["orig_points"].cpu()
    
    _, ax = plt.subplots(2, 4, figsize=(20, 8))
    ax[0, 0].axis('off')
    ax[0, 0].set_title('GT-Heatmap')
    ax[0, 0].imshow(hmap_gt)

    ax[0, 1].axis('off')
    ax[0, 1].set_title('GT-Distance Field')
    ax[0, 1].imshow(df_gt)

    ax[0, 2].axis('off')
    ax[0, 2].set_title('GT-Angle Field')
    ax[0, 2].imshow(get_flow_vis(df_gt, af_gt))

    ax[0, 3].axis('off')
    ax[0, 3].set_title('GT-Original')
    ax[0, 3].imshow(img_data_with_gt["image"].permute(1,2,0))
    ax[0, 3].scatter(orig_pt_gt[:, 0], orig_pt_gt[:, 1], marker="o", color="red", s=3)
    
    ax[1, 0].axis('off')
    ax[1, 0].set_title('KP&J Heatmap')
    ax[1, 0].imshow(jpldd_kpjhm)

    ax[1, 1].axis('off')
    ax[1, 1].set_title('Distance Field')
    ax[1, 1].imshow(jpldd_df)

    ax[1, 2].axis('off')
    ax[1, 2].set_title('Angle Field')
    ax[1, 2].imshow(get_flow_vis(jpldd_df, jpldd_af))

    ax[1, 3].axis('off')
    ax[1, 3].set_title('Original w. detected kp')
    ax[1, 3].imshow(img_torch.cpu().squeeze(0).permute(1,2,0))
    ax[1, 3].scatter(jpldd_kp[:, 0], jpldd_kp[:, 1], marker="o", color="red", s=3)
    plt.show()
    return output_model

In [11]:
from gluefactory.models import get_model

if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_built():
    device = 'mps'
else:
    device = 'cpu'
print(f"Device Used: {device}")

jpldd_conf = {
    "name": "joint_point_line_extractor",
    "max_num_keypoints": 500,  # setting for training, for eval: -1
    "timeit": True,  # override timeit: False from BaseModel
    "line_df_decoder_channels": 32,
    "line_af_decoder_channels": 32,
    "line_detection": {
        "do": True,
        "conf": {
            "num_sample": 8,
            "num_sample_strong": 150,
            "max_point_size": 1500,

        "distance_map": {
            "threshold": 0.5,
            "avg_filter_size": 13,
            "avg_filter_padding": 6,
            "avg_filter_stride": 1,
            "max_value": 2,
            "inlier_ratio": 0.8,
            "mean_value_ratio": 0.8
        },

        "mlp_conf": {
            "has_angle_field": True,
            "has_distance_field": True, 
            "num_line_samples": 30,    # number of sampled points between line endpoints
            "mlp_hidden_dims": [256, 128, 128, 64, 32],
            "pred_threshold": 0.9,
            "weights": "/local/home/fmoeller/glue-factory/outputs/training/pold2_mlp_1k_img/checkpoint_best.tar",
        }
        }
    },
    "checkpoint": "/local/home/Point-Line/outputs/training/focal_loss_experiments/rk_focal_threshDF_focal/checkpoint_best.tar"
    #"checkpoint": "/local/home/rkreft/shared_team_folder/outputs/training/rk_oxparis_focal_hard_gt/checkpoint_best.tar"
    #"checkpoint": "/local/home/rkreft/shared_team_folder/outputs/training/rk_pold2gt_oxparis_base_hard_gt/checkpoint_best.tar"
}
jpldd_model = get_model("joint_point_line_extractor")(jpldd_conf).to(device)
jpldd_model.eval()

Device Used: cuda


[10/04/2024 01:22:16 gluefactory.models.lines.pold2_mlp INFO] Successfully loaded model weights from /local/home/fmoeller/glue-factory/outputs/training/pold2_mlp_1k_img/checkpoint_best.tar
Load model parameters from checkpoint /local/home/Point-Line/outputs/training/focal_loss_experiments/rk_focal_threshDF_focal/checkpoint_best.tar


RuntimeError: Error(s) in loading state_dict for JointPointLineDetectorDescriptor:
	size mismatch for line_extractor.model.mlp.0.weight: copying a param with shape torch.Size([256, 300]) from checkpoint, the shape in current model is torch.Size([256, 60]).

## Timings
Here we run some fw passes for single images and print timings. Check settings of model and dataset to interpret results correctly. Results are given as mean seconds.

In [8]:
from pprint import pprint
import random

rand_idx = random.sample(range(0, len(ds)), 300) 

for i in rand_idx:
    img_torch = ds[i]["image"].to(device).unsqueeze(0)
    with torch.no_grad():
        output_model = jpldd_model({"image": img_torch})

timings=jpldd_model.get_current_timings(reset=True)
pprint(timings)
print(f"~FPS: {1 / (timings['total-makespan'])} using device {device}")

NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES DETECTED
NO LINES D